In [ ]:
import sys
from pathlib import Path
import time

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch related stuff
import torch 

# SBI related stuff
import sbi
from sbi.inference import SNPE

# modules
from metrics import *
from classifier import c2st
from SupportPoints import *

In [ ]:
sys.path.append('./tasks/')

Task to run (uncomment the task you wish to execute):

In [ ]:
try: 
    del prior
    del simulator
    del sur_prior
    del samples_len
    del x_obs
    del true_sample
except NameError:
    pass

task = "1D-GMM"
from GMM import *

# task = "2D-TwoMoons"
# from TwoMoons import *

# task = "3D-Sisson"
# from Sisson import *

# task = "5D-SLCP"
# from SLCP import *

# task = "6D-BayesLR"
# from BayesLR import *

# task = "10D-BerGLM"
# from BerGLM import *

Save/Load samples:

In [ ]:
iter = 1
path = f'results/{task}/{iter}'
Path(f"./{path}/samples").mkdir(parents=True, exist_ok=True)

In [ ]:
def save_sample(tensor, name):
  Path(f"{path}/samples").mkdir(parents=True, exist_ok=True)
  torch.save(tensor, f"{path}/samples/{name}.pt")

def load_sample(name):
  sample = torch.load(f"{path}/samples/{name}.pt")
  return sample

Hyper-parameters:

In [ ]:
# hyper-parameters
num_simulations = 10   # or 20 ; number of repeated experiments
num_runs = 2           # number of rounds in SNPE 
seed = 42

Run the task:

In [ ]:
# place-holders to hold the results
sl = len(samples_len)
results_c2st = torch.ones(num_simulations, sl, 4)*float('nan')
results_mmd = torch.ones(num_simulations, sl, 4)*float('nan')
results_ed = torch.ones(num_simulations, sl, 4)*float('nan')
timings = torch.ones(num_simulations, sl, 4)*float('nan')

In [ ]:
torch.manual_seed(seed)
for i in range(sl):
  for j in range(num_simulations):
    n = samples_len[i]//num_runs

    # Regular SNPE
    t0 = time.time()
    proposal = prior
    inference = SNPE(prior, density_estimator='nsf')
    for _ in range(num_runs):    
      theta = proposal.sample((n,))
      x_sim = simulator(theta)
      density_estimator = inference.append_simulations(theta, x_sim, proposal).train()
      posterior = inference.build_posterior(density_estimator)
      proposal = posterior.set_default_x(x_obs)

    sample_post1 = posterior.sample((1000,), x=x_obs)
    t1 = time.time()
    save_sample(sample_post1, f"reg_{n}_{j}")

    mmd = MMD2(true_sample[:1000,:], sample_post1)
    results_mmd[j,i,0] = mmd

    c2st_score = c2st(true_sample[:1000,], sample_post1)
    results_c2st[j,i,0] = c2st_score
   
    ed = energy_dist(true_sample[:1000,], sample_post1)
    results_ed[j,i,0] = ed

    # Surrogate (on all but 1st round)
    t2 = time.time()
    proposal = prior
    inference = SNPE(prior, density_estimator='nsf')
    for k in range(num_runs):    
      # 1st iteration - use real simulator, and train surrogate
      if k==0:  
        theta = proposal.sample((n*num_runs,))
        x_sim = simulator(theta)
          
        # train surrogate
        inference2 = SNPE(sur_prior, density_estimator='nsf')
        density_estimator = inference2.append_simulations(theta=x_sim, x=theta).train() # x and theta switch roles
        surrogate = inference2.build_posterior(density_estimator)

      # Other iterations - use surrogate/emulator instead
      else:
        theta = proposal.sample((10*n*num_runs,))
        x_sim = torch.zeros(10*n*num_runs, x_dim)
        for l in range(len(theta)):
          x_sim[l] = surrogate.sample((1,), x=theta[l,:], show_progress_bars=False)

      density_estimator = inference.append_simulations(theta, x_sim, proposal).train()
      posterior = inference.build_posterior(density_estimator)
      proposal = posterior.set_default_x(x_obs)
              
    sample_post2 = posterior.sample((1000,), x=x_obs)
    t3 = time.time()
    save_sample(sample_post2, f"sur_{n}_{j}")

    mmd = MMD2(true_sample[:1000,:], sample_post2)
    results_mmd[j,i,1] = mmd

    c2st_score = c2st(true_sample[:1000,], sample_post2)
    results_c2st[j,i,1] = c2st_score

    ed = energy_dist(true_sample[:1000,], sample_post2)
    results_ed[j,i,1] = ed

    # SP (x2)
    t4 = time.time()
    proposal = prior
    inference = SNPE(prior, density_estimator='nsf')
    for k in range(num_runs):    
      theta = proposal.sample((n*2,))
      theta_ss, _ = do_ccp(theta, n)
      theta_ss_ = constrain_points(theta_ss, proposal).reshape(-1,theta_dim)
      x_sim_ss = simulator(theta_ss_)
      density_estimator = inference.append_simulations(theta_ss_, x_sim_ss, proposal).train()
      posterior = inference.build_posterior(density_estimator)
      proposal = posterior.set_default_x(x_obs)

    sample_post3 = posterior.sample((1000,), x=x_obs)
    t5 = time.time()
    save_sample(sample_post3, f"sp_{n}_{j}")

    mmd = MMD2(true_sample[:1000,:], sample_post3)
    results_mmd[j,i,2] = mmd

    c2st_score = c2st(true_sample[:1000,], sample_post3)
    results_c2st[j,i,2] = c2st_score

    ed = energy_dist(true_sample[:1000,:], sample_post3)
    results_ed[j,i,2] = ed

    # Mixed: Surrogate + SP
    t6 = time.time()
    proposal = prior
    inference = SNPE(prior, density_estimator='nsf')
    for k in range(num_runs):    
      
      # 1st iteration - use real simulator, and train surrogate
      if k==0:  
        theta = proposal.sample((n*num_runs*2,))
        theta_ss, _ = do_ccp(theta, n*num_runs)
        theta = constrain_points(theta_ss, proposal).reshape(-1,theta_dim)
        x_sim = simulator(theta)

        # train surrogate
        inference2 = SNPE(sur_prior, density_estimator='nsf')
        density_estimator = inference2.append_simulations(theta=x_sim, x=theta).train() # x and theta switch roles
        surrogate = inference2.build_posterior(density_estimator)
          
      # Other iterations - use surrogate/emulator instead
      else:
        theta = proposal.sample((n*num_runs*10,))
        x_sim = torch.zeros(theta.shape[0], x_dim)
        for l in range(len(theta)):
          x_sim[l] = surrogate.sample((1,), x=theta[l,:], show_progress_bars=False)

      density_estimator = inference.append_simulations(theta, x_sim, proposal).train()
      posterior = inference.build_posterior(density_estimator)
      proposal = posterior.set_default_x(x_obs)

    sample_post4 = posterior.sample((1000,), x=x_obs)
    t7 = time.time()
    save_sample(sample_post4, f"mix_{n}_{j}")

    mmd = MMD2(true_sample[:1000,:], sample_post4)
    results_mmd[j,i,3] = mmd

    c2st_score = c2st(true_sample[:1000,], sample_post4)
    results_c2st[j,i,3] = c2st_score

    ed = energy_dist(true_sample[:1000,:], sample_post4)
    results_ed[j,i,3] = ed

    timings[j,i,0] = t1-t0
    timings[j,i,1] = t3-t2
    timings[j,i,2] = t5-t4
    timings[j,i,3] = t7-t6
   
    torch.save(results_mmd, f'{path}/0res_mmd.pkl')
    torch.save(results_c2st, f'{path}/1res_c2st.pkl')
    torch.save(results_ed, f'{path}/2res_ed.pkl')
    torch.save(timings, f'{path}/timings.pkl')

mmd_means = results_mmd.nanmean(dim=0)
c2st_means = results_c2st.nanmean(dim=0)
ed_means = results_ed.nanmean(dim=0)
timing_means = timings.nanmean(dim=0)

np.savetxt(f'{path}/0mmd_means.csv', mmd_means.numpy(), delimiter=",")
np.savetxt(f'{path}/1c2st_means.csv', c2st_means.numpy(), delimiter=",")
np.savetxt(f'{path}/2ed_means.csv', ed_means.numpy(), delimiter=",")
np.savetxt(f'{path}/timing_means.csv', timing_means.numpy(), delimiter=",")

mmd_vars = results_mmd.var(dim=0)
c2st_vars = results_c2st.var(dim=0)
ed_vars = results_ed.var(dim=0)
timing_vars = timings.var(dim=0)

np.savetxt(f'{path}/0mmd_vars.csv', mmd_vars.numpy(), delimiter=",")
np.savetxt(f'{path}/1c2st_vars.csv',c2st_vars.numpy(), delimiter=",")
np.savetxt(f'{path}/2ed_vars.csv',ed_vars.numpy(), delimiter=",")
np.savetxt(f'{path}/timing_vars.csv', timing_vars.numpy(), delimiter=",")